In [106]:
import mysql.connector

In [107]:
elib_db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="elibrary"
)

print(elib_db)

elib_db.autocommit = True
cursor = elib_db.cursor(buffered=True)

In [108]:
def add_book(cursor, author, title, year_of_publishing, num_of_copies):
    add_book = ("INSERT INTO books "
                "VALUES ((SELECT MAX(book_id) + 1 FROM (SELECT * FROM books) AS books_tmp),"
                " %(author)s, %(title)s, %(year_of_publishing)s, %(num_of_copies)s)")
    cursor.execute(add_book,
                   {'author': author,
                    'title': title,
                    'year_of_publishing': str(year_of_publishing),
                    'num_of_copies': str(num_of_copies)})

In [109]:
add_book(cursor, "И.М. Макаров, В.М. Лохин, С.В. Манько, М.П. Романов",
         "Искусственный интеллект и интеллектуальные системы управления", 2006, 12)

In [110]:
def search_book_by_id(cursor, id):
    search = ("SELECT * FROM books "
              "WHERE book_id = %s")
    cursor.execute(search, (str(id),))
    return cursor.fetchall()

In [111]:
print(search_book_by_id(cursor, 3))

[(3, 'Ким Д.П.', 'Теория автоматического управления, т. 1: Линейные системы', 2016, 19)]


In [154]:
def flexible_books_search(cursor,
                          author="",
                          title="",
                          year_of_publishing="",
                          author_accurate=False,
                          title_accurate=False,
                          year_sign="="):
    drop = ("DROP TABLE "
            "IF EXISTS flexible_search")
    cursor.execute(drop)

    tab = ("CREATE TABLE flexible_search AS "
           "SELECT * from books")
    cursor.execute(tab)

    if author != "":
        tab = ("DELETE FROM flexible_search "
               "WHERE NOT author ")
        if author_accurate:
            tab += "= %s"
            cursor.execute(tab, (author,))
        else:
            tab += "LIKE %s"
            cursor.execute(tab, ('%' + author + '%',))

    if title != "":
        tab = ("DELETE FROM flexible_search "
               "WHERE NOT title ")
        if title_accurate:
            tab += "= %s"
            cursor.execute(tab, (title,))
        else:
            tab += "LIKE %s"
            cursor.execute(tab, ('%' + title + '%',))

    if year_of_publishing != "":
        tab = ("DELETE FROM flexible_search "
               "WHERE NOT year_of_publishing " + year_sign + "%s")
        cursor.execute(tab, (year_of_publishing,))

    tab = "SELECT * FROM flexible_search"
    cursor.execute(tab)

    res = cursor.fetchall()

    drop = ("DROP TABLE "
            "IF EXISTS flexible_search")
    cursor.execute(drop)

    return res

In [155]:
flexible_books_search(cursor, year_of_publishing=2007, year_sign=">=")

[(1,
  'Ким Д.П.',
  'Теория игр в управлении интеллектуальными мобильными роботами - учебное пособие',
  2019,
  10),
 (2,
  'Ким Д.П.',
  'Теория автоматического управления, т. 2: Многомерные, нелинейные, оптимальные и адаптивные системы',
  2016,
  3),
 (3,
  'Ким Д.П.',
  'Теория автоматического управления, т. 1: Линейные системы',
  2016,
  19),
 (4, 'Лентин Дж.', 'Изучение робототехники с помощью Python', 2022, 5)]